In [1]:
import numpy as np
import tweepy as tw
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from itertools import islice 

# KEYS PARA CONSEGUIR OS DADOS DO TWITTER
consumer_key = 'D7CowpO9FOwoigwClR8k2MN9E'
consumer_secret = 'Oc866wtthIiOUV1iksTK7n14mEeKHUgwavJzCKP8gTMyQBKFr1'
access_token = '1542145388-xodgfVEv8WrOHNAKYtpmPUghD0nDvy5G6HTBi7P'
access_token_secret = 'w9ljBnC99rES8lT2xT50r6VIhAKl9940NkAe2Md0V3lbj'
# AUTENTICAÇÃO TWITTER
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [2]:
deputados = [["@abou_anni", "@AcacioFavacho", "@AdolfoViana_", "@adrianasounovo", "@adrianodobaldy", "@AecioNeves", 
              "@AfonsoFlorence", "@DepAfonsoHamm", "@afonso_motta", "@depaguinaldo11", "@FaleiroAirton", "@Alan_Rick", 
              "@Alceu_Moreira", "@oficialalesilva", "@AlencarBraga13", "@alessandromolon", "@AlexManentePPS", 
              "@depalexsantana", "@alefrotabrasil", "@lexandreleite", "@padilhando", "@AleSerfiotis", "@AlexisFonteyne", 
              "@Alice_Portugal"], 
             ["@alielmachado", "@dep_alinegurgel", "@AlineSleutjes", "@Altineu", "@AluisioMendesMA", "@amaronetotv", 
              "@aferreira2020", "@andrepdt12", "@DepAndreFufuca", "@AndreJanonesAdv", "@DeputadaAngela", 
              "@AntonioBrito_", "@achinaglia", "@ArnaldoJardim", "@aroldomartins", "@ArthurLira_", "@DepArthurMaia", 
              "@assis_carvalho", "@deputadoatila", "@dep_acoutinho", "@aureacarolinax", "@AureoRibeiroRJ", 
              "@Baleia_Rossi", "@dasilvabenedita", "@benesleocadiorn"], 
             ["@BetoFaroPT", "@betopereirams", "@BetoRosado", "@Biakicis", "@bibonunes1", "@bilac_pinto", 
              "@BiradoPindare", "@BocaAbertaOf", "@BohnGass", "@BoscoCosta_SE", "@boscosaraiva", "@DepBrunaFurlan", 
              "@CacaLeao", "@CamiloPSB", "@capalbertoneto", "@capitao_augusto", "@fabioabreuofc", "@capitao_wagner", 
              "@CarlaZambelli17", "@CarlosBezerraJr", "@carloschiodini", "@Carlos_Gaguim", "@carlosgomesdep", 
              "@carlosjordy", "@carlossampaio_"], 
             ["@carlosveraspt", "@CarlosZarattini", "@carmenzanotto", "@CarolDeToni", "@Cassio_4011", "@celinaleao", 
              "@CelioMouraTO", "@csilveira4599", "@CelioStudart", "@maldaner_celso", "@celsorussomanno", 
              "@depcelsosabino", "@Czmadureira", "@Charles_Federal", "@charllesevg", "@chico_dangelo", "@BrazaoChiquinho", 
              "@ToniettoChris", "@ChristianeYared", "@christinoaureo", "@dep_clarissa", "@deputadocajado", 
              "@CleberVerde10", "@dep_cel_armando", "@CoronelTadeu"], 
             ["@CovattiFilho", "@dagobertopdt", "@Drdamiaof", "@Daniel_PCdoB", "@DanielCoelho23", "@DFDanielFreitas", 
              "@DanielPMERJ", "@TrzeciakDaniel", "@DanielaWaguinho", "@danilo4010", "@depdarcidematos", 
              "@davidmirandario", "@davidbrsoares", "@DelegadoFurtado", "@EderMauroPA", "@DelegadoFreitas", 
              "@DelegadoPablo_", "@delegado_waldir", "@denisbezerra", "@diegogarciapr", "@depdimasfabiano", "@depmarcon", 
              "@Domingos_Neto", "@DomingosSavioMG", "@DrFredericoMG"], 
             ["@jaziel_dr", "@DrLeonardomt", "@drluizovando", "@zachariascalil", "@DraManato", "@dulcemiranda_to", 
              "@EdilazioJunior_", "@ediolopes", "@EdmilsonPSOL", "@eduardobarbosa_", "@eduardobismarck", 
              "@BolsonaroSP", "@EduardoBraide", "@eduardocosta14", "@Eduardo_Cury", "@eduardodafonte", 
              "@efraimfilho", "@elcionepmdb", "@elidiasborges", "@Assessoria2577", "@EliasVazGyn", "@ElmarNascimento", 
              "@emanuelzinhomt", "@enioverri", "@enricomisasi"], 
             ["@erikakokay", "@erosbiondini", "@EuclydesPetter", "@EvairdeMelo", "@roman_evandro", "@fabiofaria5555", 
              "@FabioHenriqueSE", "@depfmitidieri", "@_FabinhoRamalho", "@fabioreis1515", "@deputadofabiosc", "@f_trad", 
              "@Fausto_Pinato", "@FederalFelicio", "@felipecarreras", "@FFrancischini_", "@rigoni_felipe", 
              "@FelixMendoncaJr", "@fernandapsol", "@fernandofilhope", "@fernandogiacobo", "@fmonteiroPE", 
              "@depfernandor", "@filipebarrost", "@FlaviaArrudaDF"], 
             ["@depflaviamorais", "@Flaviano_Melo", "@flavionoguera", "@Flordelismk", "@franciscojr_go", 
              "@FrancoCartafina", "@FredCosta5133", "@freianastaciopt", "@GelsonAzevedoRJ", "@DepGenecias", 
              "@GeneralGirao", "@GenPeternelli", "@geninhozuliani", "@geovaniadesa", "@gervasiomaia", "@gilcutrim", 
              "@gilbertoabramo", "@GilbertoPSC20", "@DGildenemyr", "@gilsonmarques30", "@giovanicherini", 
              "@GiovaniFeltes", "@Glauber_Braga", "@glaustinfokus", "@gleisi"], 
             ["@greyceelias", "@zeguilherme_10", "@capitaoderrite", "@GuilhermeMussi", "@GurgelSoares", 
              "@gustavofruet", "@GustinhoRibeiro", "@guth_reis", "@ze_haroldo", "@HeitorFreireCE", "@HeitorSchuch", 
              "@heldersalomao", "@HelioCosta15", "@h_leite", "@depheliolopes", "@HenriqueFontana", "@herciliodiniz", 
              "@herculanopassos", "@HermesLaranja", "@hildorocha1513", "@deputadohiran", "@depHugoLeal", 
              "@HugoMottaPB", "@IdilvanAlencar", "@Igor_Kannario"], 
             ["@oficialigortimo", "@iracemaportela", "@bulhoesjr", "@IvanValente", "@jandira_feghali", "@jqcassol", 
              "@DepJefferson", "@jeronimogoergen", "@jesussergioac", "@JHC_40", "@DeputadoBacelar", 
              "@joaocamposdep", "@depjoaodanielpt", "@JoaoCampos", "@depjoaomaia", "@joaoromaneto", "@depjpassarinho", 
              "@JoeniaWapichana", "@jhonatan_djesus", "@joicehasselmann", "@Jorge_Braz", "@depjorgesolla", 
              "@JoseAirtonPT", "@Schiavinato_", "@guimaraes13PT"], 
             ["@JoseMedeirosMT", "@depjosenelto", "@JoseNunesDep", "@priante", "@ZeRicardoAM", "@dep_joserocha", 
              "@josiasdavitoria", "@josiasgomesba", "@josimarPL22", "@LemosFederal", "@DepJulioCesarPI", 
              "@JulioCesarRib", "@depjuliodelgado", "@JuninhodoPneu", "@cabojunioamaral", "@bozzellajr", 
              "@JuniorManoDep", "@DepJuscelino", "@kimpkat", "@LaercioFederal", "@lafayetteandrad", 
              "@laurietecantora", "@leandredaideal", "@LedaSadala", "@Dep_LeoMoraes"], 
             ["@leomotta_ofc", "@depleomonteiro", "@joseleonidas", "@LeurLomantoJr", "@lidicedamata", 
              "@lincoln_portela", "@LizianeBayer", "@LTrutis", "@LourivalGomesrj", "@lucasvgonzalez", 
              "@LucasRedecker", "@VergilioLucas77", "@Bivar1717", "@LucianoDucci", "@luciomosquini", 
              "@LuisMirandaUSA1", "@LuisTibeOficial", "@luisa_canziani", "@drluizantoniojr", "@DepLuizCarlos", 
              "@lcm_motta", "@professorLFG", "@luizlaurofilho", "@Oficialluizlima", "@LuizNishimori"],
             ["@lpbragancabr", "@luizaerundina", "@luizaogoulart", "@Luizianne13PT", "@Magdamofatto", 
              "@majorfabianadep", "@PrManuelMarcos", "@mararocha4545", "@marcelvanhattem", "@Marceloalvaroan", 
              "@marceloaro", "@caleromarcelo", "@MarceloFreixo", "@depmarcelonilo", "@marceloramosam", 
              "@AlvinoMarcio", "@marciobiolchi", "@marciojerry", "@marciolabre", "@dpmarciomarinho", 
              "@MarcoBertaiolli", "@MarcosA_Sampaio", "@marcospereira04", "@MargaretCoelho", "@JFMargarida"], 
             ["@mariadorosario", "@mariarosassp", "@DeputadaMariana", "@MariliaArraes", "@mario_heringer", 
              "@depnegromontejr", "@deputado_marlon", "@depmarrecafilho", "@marxbeltrao", "@mauricioptbrs", 
              "@mauro_bfilho", "@depmaurolopes", "@drmauronazif", "@MerlongSolano", "@Dmiguellombardi", 
              "@MiltonVieiraOfc", "@Lael_Varella", "@DepFederalMoses", "@natbonavides", "@nelsonbarbudo", 
              "@nelsonpelegrino", "@nereucrispim", "@NeriGeller", "@newton1510", "@neyleprevost"], 
             ["@depnicoletti", "@DepNilsonPinto", "@NiltoTatto", "@NivaldoAlbuque", "@odaircunhamg", 
              "@OLIVALMARQUES", "@onyxlorenzoni", "@orlandosilva", "@OrDamaso", "@OsmarTerra", "@Ossesio_Silva", 
              "@OtoniDepFederal", "@ottoalencar", "@dep_padrejoao", "@AbilioSantana_", "@DEPPASTOREURICO", 
              "@marcofeliciano", "@prisidorio", "@Patrus_Ananias", "@paulambelmonte", "@paulaodopt", "@dep_paulinho", 
              "@pauloabiackel", "@PauloAzi", "@PauloBengtson"], 
             ["@PauloMartins10", "@DeputadoFoletto", "@DF_PauloFreire", "@pauloganime", "@deppauloguedes", 
              "@DeputadoFederal", "@pauloramosdep", "@pauloteixeira13", "@PedroABezerra", "@pedroocl", 
              "@depPedroLucasF", "@Dep_PedroLupion", "@pedropaulo", "@uczai", "@pedrowestphalen", "@perpetua_acre", 
              "@pinheirinhomg", "@KatiaSastre", "@PompeodeMattos", "@deppimentel", "@marcivania65", "@profalcids", 
              "@ProfIsrael", "@professorjoziel", "@profdorinha"], 
             ["@prof_rosaneide", "@RafaelMottaRN", "@raimundocostaba", "@RaulHenry", "@ReginaldoLopes", 
              "@rejane_dias", "@RenataAbreu1919", "@renildo", "@RicardoBarrosPP", "@ricardozguidi", "@RicardoIzar", 
              "@RicardoSilvaRP", "@PSDBricardo45", "@RobertoAlvesPRB", "@robertodelucena", "@RobertoPessoaCE", 
              "@rodrigoagost", "@coelho_rodrigo", "@RodrigoCastro45", "@RodrigoMaia", "@RogerioCorreia_", 
              "@deputadopeninha", "@ronaldocarletto", "@DepRosanaValle", "@rosangelasgomes"], 
             ["@rosemodestoms", "@23rubensbueno", "@rubenspereirajr", "@RubensOtoni", "@rfalcao13", 
              "@depruycarneiro", "@samiabomfim", "@samuelmoreira", "@DepSanderson", "@SandroAlex5555", 
              "@SargentoFAHUR", "@SebaOliveirajr", "@_sergiosouza", "@depsergiotoledo", "@vidigalsergio", 
              "@DepSheridan", "@sidneyleite_", "@silascamara_", "@silviacristinax", "@Silvio_CFilho", "@sorayasantos", 
              "@DepSostenes", "@depstefano", "@SubGonzagaMG", "@tabataamaralsp"], 
             ["@TadeuAlencarPE", "@taliriapetrone", "@TerezaCrisMS", "@TerezaNelma_", "@tiago_dimas", "@TiagoMitraud", 
              "@tiriricanaweb", "@deputadotoninho", "@tuliogadelha", "@ulduricojr", "@VaidonOliveira", 
              "@ValdevanNoventa", "@DepValmir", "@Vandamilani1", "@vanderloubet", "@vanderleimacris", "@vavamartinspa", 
              "@VICENTINHOPT", "@vicentinhojr", "@Vilsondafetaemg", "@falecomvinicius", "@Vinicius_Farah", 
              "@ViniciusPoit", "@MajorVitorHugo", "@DepVitorLippi"], 
             ["@depwagnermontes", "@waldenorpereira", "@walteralvesrn", "@pradoweliton", "@wr22", "@WilsonSantiago_", 
              "@WladGarotinho", "@WolneyQueirozM", "@Dep_ZeCarlosPT", "@ZeMarioGO", "@depzeneto", "@ZeSilva_", 
              "@zevitor_mg", "@zeca_dirceu"]]

In [35]:
for item in range(len(deputados)):
    

    # CÓDIGO QUE ENTRAR NO TWITTER
    
    user = deputados[item]
    tabela_mestre = []

    for x in range(len(user)):
        tweets = tw.Cursor(api.search, q="from:" + user[x], lang="pt", tweet_mode='extended').items(1)
        users_locs = [[tweet.user.screen_name, tweet.full_text, tweet.retweet_count, tweet.favorite_count,
                       str(tweet.created_at)[0:10], str(tweet.created_at)[11:19]] for tweet in tweets]
        tabela_mestre = tabela_mestre + users_locs

    # PRINT TESTE
    print(tabela_mestre)
    print("_______________________________________________")


    # CÓDIGO QUE ENTRA NA GOOGLESHEET

    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive.file',
             "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
    client = gspread.authorize(creds)
    sheet1 = client.open('TT_dep_federais_DB').worksheet("DB")

    # CÓDIGO QUE INSERE DADOS NA GOOGLESHEET
    for x in range(len(tabela_mestre)):
        sheet1.insert_row(tabela_mestre[x], 2)
    print("_______________________________________________FIM DE CÓDIGO NÚMERO {}".format([item]))
        
    for i in range(100,1,-1):
        time.sleep(1)    
        print(i)

print("_________COMEÇANDO A APAGAR OS DADOS_________")        
        
# PEGANDO DADOS QUE ACABEI DE ADD
sheetopen = client.open("TT_dep_federais_DB").sheet1
list_of_hashes = sheetopen.get_all_records()

# FAZENDO UMA LISTA COM APENAS USUARIO E TEXTO
lista = []
for x in list_of_hashes:
    [lista.extend([k, v]) for k, v in x.items()]

words = []
multiplo = 3
for x in range(len(lista)):
    if str(x) == str(multiplo):
        words.append([lista[x - 2], lista[x].split()])
        multiplo += 12

# APAGANDO DADOS REPETIDOS
repeated = []
repeated2 = []
for x in range(len(words)):
    for y in range(len(words)):
        if words[x] == words[y]:
            repeated.append(y)
            break
for x in range(len(words)):
    for y in range(len(words)):
        if words[x] == words[y]:
            repeated2.append(y)
delete = list(list(set(repeated2) - set(repeated)) + list(set(repeated) - set(repeated2)))

delete2 = []
for x in range(len(delete)):
    delete2.append(delete[x] + 2)
    
delete2.sort()
        
delete3 = iter(delete2)
length_to_split = [100, 100, 100, 100, 100, 100, 100] 

Output = [list(islice(delete3, elem)) 
          for elem in length_to_split] 

for dados in reversed(range(len(Output))):
    if Output[dados] == []:
        pass
    else:
        for y in reversed(range(len(Output[dados]))):
            sheet1.delete_row(Output[dados][y])
        print("_______________________________________________")
        print("DADOS REPETIDOS APAGADOS")
        print("_______________________________________________FIM DE CÓDIGO NÚMERO {}".format([dados]))
        for i in range(100,1,-1):
            time.sleep(1)    
            print(i)    
print("FINALIZOU CÓDIGO__________________")
    

[['adrianodobaldy', 'Na Câmara, aprovamos socorro financeiro para esportistas durante pandemia. O projeto prevê auxílio emergencial aos profissionais autônomos da educação física e aos vinculados a clube ou entidade esportiva.\n\nhttps://t.co/2AUEmKDyNq', 0, 0, '2020-09-24', '21:53:39'], ['AfonsoFlorence', 'O PT tem propostas para a saída da crise e a reconstrução do Brasil.     \n\nhttps://t.co/2XEdfO1nv5', 0, 2, '2020-09-26', '20:59:34'], ['DepAfonsoHamm', 'Com transparência, respeito e diálogo tenho certeza que a caminhada será vitoriosa! Um forte abraço! #trabalhoeresultado #eleicoes2020 #progressistas', 0, 3, '2020-09-27', '10:49:32'], ['FaleiroAirton', 'RT @andretrig: Boiada à vista! Salles quer votar na 2ª feira - em regime de urgência - revogação da Resolução 303 do Conama que protege Áre…', 3959, 0, '2020-09-26', '10:09:18'], ['Alan_Rick', 'RT @embaixadoryossi: Agradeço presidente @jairmessiasbolsonaro pela mensagem fraterna em razão do Yom Kipur, Dia do Perdão, data important

KeyboardInterrupt: 

In [ ]:
print(Output)

In [28]:
# PEGANDO DADOS QUE ACABEI DE ADD
sheetopen = client.open("TT_dep_federais_DB").sheet1
list_of_hashes = sheetopen.get_all_records()

# FAZENDO UMA LISTA COM APENAS USUARIO E TEXTO
lista = []
for x in list_of_hashes:
    [lista.extend([k, v]) for k, v in x.items()]

words = []
multiplo = 3
for x in range(len(lista)):
    if str(x) == str(multiplo):
        words.append([lista[x - 2], lista[x].split()])
        multiplo += 12

# APAGANDO DADOS REPETIDOS
repeated = []
repeated2 = []
for x in range(len(words)):
    for y in range(len(words)):
        if words[x] == words[y]:
            repeated.append(y)
            break
for x in range(len(words)):
    for y in range(len(words)):
        if words[x] == words[y]:
            repeated2.append(y)
delete = list(list(set(repeated2) - set(repeated)) + list(set(repeated) - set(repeated2)))

delete2 = []
for x in range(len(delete)):
    delete2.append(delete[x] + 2)
delete2.sort()

In [29]:
print(delete2)


[438, 439, 440, 441, 445, 452, 453, 454, 455, 456, 457, 460, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 474, 475, 476, 477, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558]


In [30]:
delete3 = iter(delete2)
length_to_split = [100, 100, 100, 100, 100, 100] 

In [31]:
Output = [list(islice(delete3, elem)) 
          for elem in length_to_split] 

In [32]:
print(len(Output))

6


In [33]:
print(len(delete2))

100


In [34]:

#for x in reversed(range(len(delete2))):
#    sheet1.delete_row(delete2[x])

#delete3 = iter(delete2)
#length_to_split = [100, 100, 100, 100, 100, 100] 

#Output = [list(islice(delete3, elem)) 
 #         for elem in length_to_split] 

for dados in reversed(range(len(Output))):
    if Output[dados] == []:
        pass
    else:
        for y in reversed(range(len(Output[dados]))):
            sheet1.delete_row(Output[dados][y])
        print("_______________________________________________")
        print("DADOS REPETIDOS APAGADOS")
        print("_______________________________________________FIM DE CÓDIGO NÚMERO {}".format([dados]))
        for i in range(100,1,-1):
            time.sleep(1)    
            print(i)

<ipython-input-34-e2628a87ba2f>:15: DeprecationWarning: Worksheet.delete_row() is deprecated, Please use `Worksheet.delete_rows()` instead.
  sheet1.delete_row(Output[dados][y])


_______________________________________________
DADOS REPETIDOS APAGADOS
_______________________________________________FIM DE CÓDIGO NÚMERO [0]
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2


In [ ]:
["@maldaner_celso"
, "@celsorussomanno"
, "@depcelsosabino"
, "@Czmadureira"
, "@Charles_Federal"
, "@charllesevg"
, "@chico_dangelo"
, "@BrazaoChiquinho"
, "@ToniettoChris"
, "@ChristianeYared"
, "@christinoaureo"
, "@dep_clarissa"
, "@deputadocajado"
, "@CleberVerde10"
, "@dep_cel_armando"
, "@CoronelTadeu"
, "@CovattiFilho"
, "@dagobertopdt"
, "@Drdamiaof"
, "@Daniel_PCdoB"
, "@DanielCoelho23"
, "@DFDanielFreitas"
, "@DanielPMERJ"
, "@TrzeciakDaniel"
, "@DanielaWaguinho"
, "@danilo4010"
, "@depdarcidematos"
, "@davidmirandario"
], ["@davidbrsoares"
, "@DelegadoFurtado"
, "@EderMauroPA"
, "@DelegadoFreitas"
, "@DelegadoPablo_"
, "@delegado_waldir"
, "@denisbezerra"
, "@diegogarciapr"
, "@depdimasfabiano"
, "@depmarcon"
, "@Domingos_Neto"
, "@DomingosSavioMG"
, "@DrFredericoMG"
, "@jaziel_dr"
, "@DrLeonardomt"
, "@drluizovando"
, "@zachariascalil"
, "@DraManato"
, "@dulcemiranda_to"
, "@EdilazioJunior_"
, "@ediolopes"
, "@EdmilsonPSOL"
, "@eduardobarbosa_"
, "@eduardobismarck"
, "@BolsonaroSP"
, "@EduardoBraide"
, "@eduardocosta14"
, "@Eduardo_Cury"
], ["@eduardodafonte"
, "@efraimfilho"
, "@elcionepmdb"
, "@elidiasborges"
, "@Assessoria2577"
, "@EliasVazGyn"
, "@ElmarNascimento"
, "@emanuelzinhomt"
, "@enioverri"
, "@enricomisasi"
, "@erikakokay"
, "@erosbiondini"
, "@EuclydesPetter"
, "@EvairdeMelo"
, "@roman_evandro"
, "@fabiofaria5555"
, "@FabioHenriqueSE"
, "@depfmitidieri"
, "@_FabinhoRamalho"
, "@fabioreis1515"
, "@deputadofabiosc"
, "@f_trad"
, "@Fausto_Pinato"
, "@FederalFelicio"
, "@felipecarreras"
, "@FFrancischini_"
, "@rigoni_felipe"
, "@FelixMendoncaJr"
], ["@fernandapsol"
, "@fernandofilhope"
, "@fernandogiacobo"
, "@fmonteiroPE"
, "@depfernandor"
, "@filipebarrost"
, "@FlaviaArrudaDF"
, "@depflaviamorais"
, "@Flaviano_Melo"
, "@flavionoguera"
, "@Flordelismk"
, "@franciscojr_go"
, "@FrancoCartafina"
, "@FredCosta5133"
, "@freianastaciopt"
, "@GelsonAzevedoRJ"
, "@DepGenecias"
, "@GeneralGirao"
, "@GenPeternelli"
, "@geninhozuliani"
, "@geovaniadesa"
, "@gervasiomaia"
, "@gilcutrim"
, "@gilbertoabramo"
, "@GilbertoPSC20"
, "@DGildenemyr"
, "@gilsonmarques30"
, "@giovanicherini"
], ["@GiovaniFeltes"
, "@Glauber_Braga"
, "@glaustinfokus"
, "@gleisi"
, "@greyceelias"
, "@zeguilherme_10"
, "@capitaoderrite"
, "@GuilhermeMussi"
, "@GurgelSoares"
, "@gustavofruet"
, "@GustinhoRibeiro"
, "@guth_reis"
, "@ze_haroldo"
, "@HeitorFreireCE"
, "@HeitorSchuch"
, "@heldersalomao"
, "@HelioCosta15"
, "@h_leite"
, "@depheliolopes"
, "@HenriqueFontana"
, "@herciliodiniz"
, "@herculanopassos"
, "@HermesLaranja"
, "@hildorocha1513"
, "@deputadohiran"
, "@depHugoLeal"
, "@HugoMottaPB"
, "@IdilvanAlencar"
], ["@Igor_Kannario"
, "@oficialigortimo"
, "@iracemaportela"
, "@bulhoesjr"
, "@IvanValente"
, "@jandira_feghali"
, "@jqcassol"
, "@DepJefferson"
, "@jeronimogoergen"
, "@jesussergioac"
, "@JHC_40"
, "@DeputadoBacelar"
, "@joaocamposdep"
, "@depjoaodanielpt"
, "@JoaoCampos"
, "@depjoaomaia"
, "@joaoromaneto"
, "@depjpassarinho"
, "@JoeniaWapichana"
, "@jhonatan_djesus"
, "@joicehasselmann"
, "@Jorge_Braz"
, "@depjorgesolla"
, "@JoseAirtonPT"
, "@Schiavinato_"
, "@guimaraes13PT"
, "@JoseMedeirosMT"
, "@depjosenelto"
], ["@JoseNunesDep"
, "@priante"
, "@ZeRicardoAM"
, "@dep_joserocha"
, "@josiasdavitoria"
, "@josiasgomesba"
, "@josimarPL22"
, "@LemosFederal"
, "@DepJulioCesarPI"
, "@JulioCesarRib"
, "@depjuliodelgado"
, "@JuninhodoPneu"
, "@cabojunioamaral"
, "@bozzellajr"
, "@JuniorManoDep"
, "@DepJuscelino"
, "@kimpkat"
, "@LaercioFederal"
, "@lafayetteandrad"
, "@laurietecantora"
, "@leandredaideal"
, "@LedaSadala"
, "@Dep_LeoMoraes"
, "@leomotta_ofc"
, "@depleomonteiro"
, "@joseleonidas"
, "@LeurLomantoJr"
, "@lidicedamata"
], ["@lincoln_portela"
, "@LizianeBayer"
, "@LTrutis"
, "@LourivalGomesrj"
, "@lucasvgonzalez"
, "@LucasRedecker"
, "@VergilioLucas77"
, "@Bivar1717"
, "@LucianoDucci"
, "@luciomosquini"
, "@LuisMirandaUSA1"
, "@LuisTibeOficial"
, "@luisa_canziani"
, "@drluizantoniojr"
, "@DepLuizCarlos"
, "@lcm_motta"
, "@professorLFG"
, "@luizlaurofilho"
, "@Oficialluizlima"
, "@LuizNishimori"
, "@lpbragancabr"
, "@luizaerundina"
, "@luizaogoulart"
, "@Luizianne13PT"
, "@Magdamofatto"
, "@majorfabianadep"
, "@PrManuelMarcos"
, "@mararocha4545"
], ["@marcelvanhattem"
, "@Marceloalvaroan"
, "@marceloaro"
, "@caleromarcelo"
, "@MarceloFreixo"
, "@depmarcelonilo"
, "@marceloramosam"
, "@AlvinoMarcio"
, "@marciobiolchi"
, "@marciojerry"
, "@marciolabre"
, "@dpmarciomarinho"
, "@MarcoBertaiolli"
, "@MarcosA_Sampaio"
, "@marcospereira04"
, "@MargaretCoelho"
, "@JFMargarida"
, "@mariadorosario"
, "@mariarosassp"
, "@DeputadaMariana"
, "@MariliaArraes"
, "@mario_heringer"
, "@depnegromontejr"
, "@deputado_marlon"
, "@depmarrecafilho"
, "@marxbeltrao"
, "@mauricioptbrs"
, "@mauro_bfilho"
], ["@depmaurolopes"
, "@drmauronazif"
, "@MerlongSolano"
, "@Dmiguellombardi"
, "@MiltonVieiraOfc"
, "@Lael_Varella"
, "@DepFederalMoses"
, "@natbonavides"
, "@nelsonbarbudo"
, "@nelsonpelegrino"
, "@nereucrispim"
, "@NeriGeller"
, "@newton1510"
, "@neyleprevost"
, "@depnicoletti"
, "@DepNilsonPinto"
, "@NiltoTatto"
, "@NivaldoAlbuque"
, "@odaircunhamg"
, "@OLIVALMARQUES"
, "@onyxlorenzoni"
, "@orlandosilva"
, "@OrDamaso"
, "@OsmarTerra"
, "@Ossesio_Silva"
, "@OtoniDepFederal"
, "@ottoalencar"
, "@dep_padrejoao"
], ["@AbilioSantana_"
, "@DEPPASTOREURICO"
, "@marcofeliciano"
, "@prisidorio"
, "@Patrus_Ananias"
, "@paulambelmonte"
, "@paulaodopt"
, "@dep_paulinho"
, "@pauloabiackel"
, "@PauloAzi"
, "@PauloBengtson"
, "@PauloMartins10"
, "@DeputadoFoletto"
, "@DF_PauloFreire"
, "@pauloganime"
, "@deppauloguedes"
, "@DeputadoFederal"
, "@pauloramosdep"
, "@pauloteixeira13"
, "@PedroABezerra"
, "@pedroocl"
, "@depPedroLucasF"
, "@Dep_PedroLupion"
, "@pedropaulo"
, "@uczai"
, "@pedrowestphalen"
, "@perpetua_acre"
, "@pinheirinhomg"
], ["@KatiaSastre"
, "@PompeodeMattos"
, "@deppimentel"
, "@marcivania65"
, "@profalcids"
, "@ProfIsrael"
, "@professorjoziel"
, "@profdorinha"
, "@prof_rosaneide"
, "@RafaelMottaRN"
, "@raimundocostaba"
, "@RaulHenry"
, "@ReginaldoLopes"
, "@rejane_dias"
, "@RenataAbreu1919"
, "@renildo"
, "@RicardoBarrosPP"
, "@ricardozguidi"
, "@RicardoIzar"
, "@RicardoSilvaRP"
, "@PSDBricardo45"
, "@RobertoAlvesPRB"
, "@robertodelucena"
, "@RobertoPessoaCE"
, "@rodrigoagost"
, "@coelho_rodrigo"
, "@RodrigoCastro45"
, "@RodrigoMaia"
], ["@RogerioCorreia_"
, "@deputadopeninha"
, "@ronaldocarletto"
, "@DepRosanaValle"
, "@rosangelasgomes"
, "@rosemodestoms"
, "@23rubensbueno"
, "@rubenspereirajr"
, "@RubensOtoni"
, "@rfalcao13"
, "@depruycarneiro"
, "@samiabomfim"
, "@samuelmoreira"
, "@DepSanderson"
, "@SandroAlex5555"
, "@SargentoFAHUR"
, "@SebaOliveirajr"
, "@_sergiosouza"
, "@depsergiotoledo"
, "@vidigalsergio"
, "@DepSheridan"
, "@sidneyleite_"
, "@silascamara_"
, "@silviacristinax"
, "@Silvio_CFilho"
, "@sorayasantos"
, "@DepSostenes"
, "@depstefano"
], ["@SubGonzagaMG"
, "@tabataamaralsp"
, "@TadeuAlencarPE"
, "@taliriapetrone"
, "@TerezaCrisMS"
, "@TerezaNelma_"
, "@tiago_dimas"
, "@TiagoMitraud"
, "@tiriricanaweb"
, "@deputadotoninho"
, "@tuliogadelha"
, "@ulduricojr"
, "@VaidonOliveira"
, "@ValdevanNoventa"
, "@DepValmir"
, "@Vandamilani1"
, "@vanderloubet"
, "@vanderleimacris"
, "@vavamartinspa"
, "@VICENTINHOPT"
, "@vicentinhojr"
, "@Vilsondafetaemg"
, "@falecomvinicius"
, "@Vinicius_Farah"
, "@ViniciusPoit"
, "@MajorVitorHugo"
, "@DepVitorLippi"
, "@depwagnermontes"
], ["@waldenorpereira"
, "@walteralvesrn"
, "@pradoweliton"
, "@wr22"
, "@WilsonSantiago_"
, "@WladGarotinho"
, "@WolneyQueirozM"
, "@Dep_ZeCarlosPT"
, "@ZeMarioGO"
, "@depzeneto"
, "@ZeSilva_"
, "@zevitor_mg"
, "@zeca_dirceu" ]]